In [1]:
import tweepy
from twarc import Twarc
import pandas as pd
import json

with open('creds.json') as json_file:
    creds = json.load(json_file)

## Example tweets and retweets according to https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/api-reference/get-statuses-user_timeline

In [2]:
twarc_object = Twarc(creds["client_key"], 
                     creds["client_secret"], 
                     creds["access_token"], 
                     creds["access_token_secret"])

In [3]:
user = "@DEFENSORIAEC"

user = user.replace("@","")

user

'DEFENSORIAEC'

In [4]:

tweets = []
retweets = []
for tweet in twarc_object.timeline(screen_name=user,
                                   since_id = None,
                                   max_pages=2):
    current_id = tweet["id"]
    # getting tweets
    if ('RT @' not in tweet["full_text"]): 
        tweets.append(tweet)
    # getting retweets
    else:
        retweets.append(tweet)


In [5]:
texts = []
for idx, tweet in enumerate(tweets):
    texts.append(tweet["full_text"])

In [6]:
pd.DataFrame(texts)

,0
0,Noticia | Compartimos publicación de la Asocia...
1,"... Dr. Freddy Carrión Intriago: ""Somos el seg..."
2,... lo cual no permite conocer cuántas vacunas...
3,"Dr. Freddy Carrión Intriago: ""La información e..."
4,"... Además, tuvimos otras denuncias al respect..."
...,...
346,"Video | Dr. Freddy Carrión Intriago, expresa p..."
347,@nkdici @lahoraecuador @eluniversocom @Expreso...
348,@fredarri @lahoraecuador @eluniversocom @TVCEl...
349,#ATENCIÓN | La #DefensoríaDelPueblo exhorta al...


## Example getting mentions according to https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/api-reference/get-statuses-mentions_timeline


para este twarc no sirve, entonces mejor usar tweepy search usando los operadoers https://developer.twitter.com/en/docs/twitter-api/v1/rules-and-filtering/search-operators

https://docs.tweepy.org/en/latest/api.html


toca verificar que esta opcion n

In [38]:
f'to:{user} OR @{user}'

'to:DEFENSORIAEC OR @DEFENSORIAEC'

In [33]:
import tweepy

In [34]:
auth = tweepy.AppAuthHandler(creds["client_key"], creds["client_secret"])
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [35]:
tweets = api.search(q=f'to:{user} OR @{user}',  tweet_mode="extended")

In [36]:
texts = []
for idx, tweet in enumerate(tweets):
    texts.append(tweet._json["full_text"])

In [37]:
pd.DataFrame(texts)

,0
0,RT @recategorizados: #URGENTE Vamos a estar at...
1,RT @recategorizados: #URGENTE El Ministerio de...
2,Fue sacado de su trabajo en plena pandemia pre...
3,@BiessEcuador @DEFENSORIAEC @teleamazonasec @t...
4,RT @DonNazaQ: CONDENEMOS LAS EXPRESIONES RACIS...
5,RT @apnacc: Niños en lucha contra el cáncer se...
6,Que está pasando aquí? Esta demanda está desde...
7,@CancilleriaCol @DEFENSORIAEC Que tristeza una...
8,@marlontenecela @CynthiaViteri6 @alcaldiagye @...
9,RT @recategorizados: #URGENTE El Ministerio de...
